# Imports

In [ ]:
!pip install -i https://pypi.clarin-pl.eu lpmn_client

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import re, string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.losses import sparse_categorical_crossentropy
from lpmn_client.src.requester import Requester
import zipfile
from tensorflow.keras.utils import to_categorical
import xml.etree.ElementTree as ET
import numbers
import decimal
import tensorflow as tf

# Data loading

In [ ]:
data_path = 'data/citations.csv'
tfidf_features_path = 'data/tfidf_all_features.csv'

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 120)

df = pd.read_csv(data_path)
df = df.dropna()
df = df.reset_index(drop=True)
# df.info()

classes_names = {0: 'Fałsz', 1: 'Prawda', 2: "Manipulacja", 3: "Nieweryfikowalne"}
df['label'].replace({'Fałsz': 0, 'Prawda': 1, 'Manipulacja': 2, 'Nieweryfikowalne': 3}, inplace=True)
df = df[(df['label'] == 0) | (df['label'] == 1)]
# X.head()
# y.head(10)
df

# Feature creation

In [ ]:
def count_uppercase_letters(text):
    count = sum([1 for char in text if char.isupper()])
    return (count / (len(text) - text.count(' '))) * 100


def count_exclamation_marks(text):
    count = text.count('!')
    return (count / (len(text) - text.count(' '))) * 100


def count_question_marks(text):
    count = text.count('?')
    return (count / (len(text) - text.count(' '))) * 100


def count_quotation_marks(text):
    count = text.count('"')
    return (count / (len(text) - text.count(' '))) * 100


def count_punctuation(text):
    count = sum([1 for char in text if char in string.punctuation])
    return (count / (len(text) - text.count(' '))) * 100


def count_text_length(text):
    return len(text) - text.count(' ')


def get_sentiment(text):
    requester = Requester('241393@student.pwr.edu.pl')
    lpmn_query = 'any2txt|wcrft2|wsd|ccl_emo({"lang":"polish"})|ccl_emo_stats({' \
                 '"lang":"polish", "split_paragraphs": false})'

    string_ids = requester.upload_strings([text])
    response = requester.process_query(lpmn_query, [string_id.text for string_id in string_ids])
    requester.download_response(response[0], './sentiment.zip')

    try:
        with zipfile.ZipFile('sentiment.zip', 'r') as archive:
            with archive.open(archive.namelist()[0]) as data:
                df = pd.read_csv(data, sep=';')

        sentiment_value = sum([int(entry) for entry in df['Polarity'].values if
                               (type(entry) == str and entry.isnumeric()) or isinstance(entry, (int, float, complex))])
    except Exception as e:
        print(e)
        sentiment_value = 0

    return sentiment_value


def count_positive_words():
    try:
        with zipfile.ZipFile('sentiment.zip', 'r') as archive:
            with archive.open(archive.namelist()[0]) as data:
                df = pd.read_csv(data, sep=';')

        positive_words = (sum([1 for entry in df['Polarity'].values if
                               (type(entry) == str and entry.isnumeric() or isinstance(entry,
                                                                                       (int, float, complex))) and int(
                                   entry) > 0]) / len(df['Polarity'])) * 100
    except Exception as e:
        print(e)
        positive_words = 0

    return positive_words


def count_negative_words():
    try:
        with zipfile.ZipFile('sentiment.zip', 'r') as archive:
            with archive.open(archive.namelist()[0]) as data:
                df = pd.read_csv(data, sep=';')

        negative_words = (sum([1 for entry in df['Polarity'].values if
                               (type(entry) == str and entry.isnumeric() or isinstance(entry,
                                                                                       (int, float, complex))) and int(
                                   entry) < 0]) / len(df['Polarity'])) * 100
    except Exception as e:
        print(e)
        negative_words = 0

    return negative_words

In [ ]:
# Run only if there is a need to regenerate all the features. Otherwise, go to Loading features.
df['uppercase%'] = df['content'].apply(lambda x: count_uppercase_letters(x))
df['exclamation_mark%'] = df['content'].apply(lambda x: count_exclamation_marks(x))
df['question_mark%'] = df['content'].apply(lambda x: count_question_marks(x))
df['quotation_mark%'] = df['content'].apply(lambda x: count_quotation_marks(x))
df['punctuation%'] = df['content'].apply(lambda x: count_punctuation(x))
df['length'] = df['content'].apply(lambda x: count_text_length(x))

rows_list = []
for index, row in df.iterrows():
    dictionary = {'sentiment': get_sentiment(df.at[index, 'content']),
                  'positive_words%': count_positive_words(),
                  'negative_words%': count_negative_words()}

    rows_list.append(dictionary)
    print(f'Index: {index}')

df = pd.concat([df.reset_index(drop=True), pd.DataFrame(rows_list).reset_index(drop=True)], axis=1)

In [ ]:
df.head()

In [ ]:
df.to_csv(tfidf_features_path)

# Text preprocessing

In [ ]:
def remove_punctuation(text):
    no_punctuation_text = "".join([char for char in text if char not in string.punctuation])
    return no_punctuation_text


def lemmatize(text):
    requester = Requester('241393@student.pwr.edu.pl')
    lpmn_query = 'any2txt|wcrft2({"guesser":false, "morfeusz2":true})'

    string_ids = requester.upload_strings([text])
    response = requester.process_query(lpmn_query, [id.text for id in string_ids])
    requester.download_response(response[0], './lem.zip')

    lemmatized_text = None
    try:
        with zipfile.ZipFile('lem.zip', 'r') as archive:
            data = archive.read(archive.namelist()[0])
            lemmatized_text = [word.text for word in ET.fromstring(data).findall('chunk/sentence/tok/lex/base')]
    except Exception as e:
        print(e)

    return lemmatized_text


def tokenize(text):
    tokens = re.split('\W+', text)
    text = [word for word in tokens]
    return text


def remove_stopwords(text):
    requester = Requester('241393@student.pwr.edu.pl')
    lpmn_query = 'any2txt|morphoDita|dir|termopl2({\"mw\":false,\"sw\":\"/resources/termopl/termopl_sw.txt\",' \
                 '\"cp\":\"/resources/termopl/termopl_cp.txt\"}) '

    string_ids = requester.upload_strings([text])
    response = requester.process_query(lpmn_query, [string_id.text for string_id in string_ids])
    requester.download_response(response[0], './no_stopwords.zip')

    lemmatized_text_without_stopwords = None
    try:
        with zipfile.ZipFile('no_stopwords.zip', 'r') as archive:
            with archive.open(archive.namelist()[0]) as data:
                column_names = ['idx', 'ranking', 'output_phrase', 'original_phrase', 'c-value',
                                'length', 'freq_s', 'freq_in', 'context']
                df = pd.read_csv(data, sep='\t', names=column_names)

        lemmatized_text_without_stopwords = df['output_phrase'].tolist()
    except IndexError as e:
        print(e)
    except Exception as e:
        print(e)
        
    return lemmatized_text_without_stopwords


#TODO when we decide what how to preprocess text
# now it is just copy-paste from online course
def clean_text(text):
    no_punctuation_text = remove_punctuation(text)
    cleaned_text = lemmatize(no_punctuation_text)
    return cleaned_text

In [ ]:
# Took only 5 first elements to demonstrate the output
df['lemmatized'] = df['content'][:5].apply(lambda x: lemmatize(x))
df.head()

In [ ]:
df['lemmatized_without_stopwords'] = df['content'][:5].apply(lambda x: remove_stopwords(x))
df.head()

In [ ]:
df['clean_text'] = df['content'].apply(lambda x: clean_text(x))
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df[['clean_text', 'uppercase%', 'exclamation_mark%', 'question_mark%', 'quotation_mark%', 'punctuation%',
        'length', 'sentiment', 'positive_words%', 'negative_words%']], df['label'], test_size=0.25)

In [ ]:
X_train

## Vectorization

### N-gram

In [ ]:
# range = (2, 2)

# ngram_vect = CountVectorizer(ngram_range=range)
# X_ngram = ngram_vect.fit_transform(X['content'])

# X_ngram_df = pd.DataFrame(X_ngram.toarray())
# X_ngram_df.columns = ngram_vect.get_feature_names()
# X_ngram_df

### TF-IDF

In [ ]:
clean_joined = df['clean_text'].apply(lambda x: ' '.join(x))
X_train_clean_joined = X_train['clean_text'].apply(lambda x: ' '.join(x))
X_test_clean_joined = X_test['clean_text'].apply(lambda x: ' '.join(x))

tfidf_vect = TfidfVectorizer(ngram_range=(1, 3))
tfidf_vect_fit = tfidf_vect.fit(X_train_clean_joined)

tfidf_train = tfidf_vect_fit.transform(X_train_clean_joined)
tfidf_test = tfidf_vect_fit.transform(X_test_clean_joined)

X_train_vect = pd.concat([X_train[
                              ['uppercase%', 'exclamation_mark%', 'question_mark%', 'quotation_mark%', 'punctuation%',
                               'length', 'sentiment', 'positive_words%', 'negative_words%']].reset_index(drop=True),
                          pd.DataFrame(tfidf_train.toarray())], axis=1)

X_test_vect = pd.concat([X_test[
                             ['uppercase%', 'exclamation_mark%', 'question_mark%', 'quotation_mark%', 'punctuation%',
                              'length', 'sentiment', 'positive_words%', 'negative_words%']].reset_index(drop=True),
                         pd.DataFrame(tfidf_test.toarray())], axis=1)

In [ ]:
# Saving training and testing sets
X_train_vect.to_csv(f'{content_root}/X_train_tfidf.csv', index=False, header=True)
X_test_vect.to_csv(f'{content_root}/X_test_tfidf.csv', index=False, header=True)
y_train.to_csv(f'{content_root}/y_train_tfidf.csv', index=False, header=True)
y_test.to_csv(f'{content_root}/y_test_tfidf.csv', index=False, header=True)

In [ ]:
# Saving all the features (only for GridSearchCV)
X_tfidf = tfidf_vect.fit_transform(clean_joined)
X_tfidf_feat = pd.concat([df[['uppercase%', 'exclamation_mark%', 'question_mark%', 'quotation_mark%', 'punctuation%',
                              'length', 'sentiment', 'positive_words%', 'negative_words%']].reset_index(drop=True),
                          pd.DataFrame(X_tfidf.toarray())], axis=1)

X_tfidf_feat.to_csv(tfidf_features_path)

In [ ]:
# X_test_vect[0].toarray()

In [ ]:
# Loading features
X_tfidf_feat = pd.read_csv(tfidf_features_path)

# Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=1410)
param = {
    'max_depth': [80, 90, 100, 110, None],
    'max_features': [0.2, None],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1, return_train_score=True)
gs_fit = gs.fit(X_tfidf_feat, data['label'])

In [ ]:
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

# Model nr 2

# Deep Learning

In [63]:
#read in clean text
df = pd.read_csv('data/clean.csv')

In [64]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size=0.25)

In [65]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

padding_length = len(max(X_train_seq, key=len))

X_train_seq_pad = pad_sequences(X_train_seq, padding_length)
X_test_seq_pad = pad_sequences(X_test_seq, padding_length)



In [ ]:
X_train

## RNN model

In [ ]:
X_train_seq_pad

In [83]:
batch_size = 32

model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, 32))
model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3, recurrent_initializer='glorot_uniform', return_sequences=True))
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.3, recurrent_initializer='glorot_uniform'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 32)          239808    
_________________________________________________________________
lstm_11 (LSTM)               (None, None, 64)          24832     
_________________________________________________________________
lstm_12 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dense_34 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 130       
Total params: 279,298
Trainable params: 279,298
Non-trainable params: 0
_________________________________________________________________


In [84]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
early_stopping = EarlyStopping(monitor='loss', patience=3)

y_train_cat = to_categorical(y_train, 2)
y_test_cat = to_categorical(y_test, 2)

history = model.fit(X_train_seq_pad, y_train_cat, validation_data=(X_test_seq_pad, y_test_cat), batch_size=batch_size, epochs=10, callbacks=[early_stopping])

Epoch 1/10
84/84 [==============================] - 36s 330ms/step - loss: 0.6164 - accuracy: 0.7213 - val_loss: 0.5969 - val_accuracy: 0.7167
Epoch 2/10
84/84 [==============================] - 26s 314ms/step - loss: 0.5890 - accuracy: 0.7263 - val_loss: 0.5992 - val_accuracy: 0.7167
Epoch 3/10
84/84 [==============================] - 28s 330ms/step - loss: 0.5763 - accuracy: 0.7166 - val_loss: 0.5958 - val_accuracy: 0.7122
Epoch 4/10
84/84 [==============================] - 26s 313ms/step - loss: 0.3876 - accuracy: 0.8271 - val_loss: 0.7699 - val_accuracy: 0.6920
Epoch 5/10
84/84 [==============================] - 26s 307ms/step - loss: 0.1977 - accuracy: 0.9199 - val_loss: 0.9658 - val_accuracy: 0.6708
Epoch 6/10
84/84 [==============================] - 27s 324ms/step - loss: 0.1048 - accuracy: 0.9643 - val_loss: 1.2464 - val_accuracy: 0.6473
Epoch 7/10
84/84 [==============================] - 21s 252ms/step - loss: 0.0643 - accuracy: 0.9793 - val_loss: 1.5498 - val_accuracy: 0.6495

### Prediction presentation

In [86]:

for numb in range(100, 140):
    print('Treść wypowiedzi:')
    print(X_test.iloc[numb])
    print('Klasyfikacja: ', y_test.iloc[numb])
    prediction = model.predict(np.array([X_test_seq[numb]]))
    print('Predykcja: ', [round(prc, 2) for prc in prediction[0]])
    print('================================================')


Treść wypowiedzi:
['i', 'co', 'by', 'być', 'tutaj', 'nie', 'powiedzieć', 'złe', 'bo', 'zawsze', 'krytyk', 'się', 'znajda', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'to', 'Polska', 'w', 'moment', 'start', 'w', '89', 'rok', 'była', 'na', 'poziom', 'Białoruś', 'i', 'Ukraina', 'ukraina', 'czyli', 'przeciętny', 'Polak', 'wytwarzać', 'tyle', 'to', 'dochód', 'narodowy', 'ile', 'Białorusin', 'i', 'Ukrainiec', 'tam', 'minimalny', 'różnica', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'dolar'

In [ ]:
# def rnn_text_classification(text):
#     prediction = model.predict(np.array([X_test_seq[numb]]))
#     prediction = [round(prc, 2) for prc in prediction[0]]
    
#     return prediction


In [87]:
result = tf.argmax(model.predict_on_batch(tf.expand_dims(X_test_seq_pad[16], 0)), axis=1)

## Artificial Neural Network

In [94]:
padding_length = len(max(X_train_seq, key=len))

In [116]:
model = Sequential()
model.add(Dense(padding_length, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))


In [117]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

In [105]:
X_train_seq_pad = pad_sequences(X_train_seq, padding_length)
X_test_seq_pad = pad_sequences(X_test_seq, padding_length)

array([[   0,    0,    0, ...,  702,   24,  322],
       [   0,    0,    0, ...,  703,   58, 3846],
       [   0,    0,    0, ...,    7,   53, 1062],
       ...,
       [   0,    0,    0, ...,    5, 3125, 3170],
       [   0,    0,    0, ...,   15, 1781,  471],
       [   0,    0,    0, ...,    4,  266, 1813]], dtype=int32)

In [118]:
history = model.fit(X_train_seq_pad, y_train_cat, validation_data=(X_test_seq_pad, y_test_cat), batch_size=batch_size, epochs=10, callbacks=[early_stopping])

Epoch 1/10
84/84 [==============================] - 3s 7ms/step - loss: 0.2993 - accuracy: 0.6957 - val_loss: 0.2795 - val_accuracy: 0.7167
Epoch 2/10
84/84 [==============================] - 0s 4ms/step - loss: 0.2613 - accuracy: 0.7089 - val_loss: 0.2055 - val_accuracy: 0.7100
Epoch 3/10
84/84 [==============================] - 0s 4ms/step - loss: 0.2169 - accuracy: 0.7188 - val_loss: 0.2016 - val_accuracy: 0.7167
Epoch 4/10
84/84 [==============================] - 0s 4ms/step - loss: 0.2116 - accuracy: 0.7176 - val_loss: 0.2023 - val_accuracy: 0.7167
Epoch 5/10
84/84 [==============================] - 0s 4ms/step - loss: 0.1956 - accuracy: 0.7294 - val_loss: 0.2037 - val_accuracy: 0.7167
Epoch 6/10
84/84 [==============================] - 0s 4ms/step - loss: 0.1990 - accuracy: 0.7167 - val_loss: 0.2024 - val_accuracy: 0.7178
Epoch 7/10
84/84 [==============================] - 0s 4ms/step - loss: 0.1955 - accuracy: 0.7317 - val_loss: 0.2027 - val_accuracy: 0.7167
Epoch 8/10
84/84 [==

In [119]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             (None, 178)               31862     
_________________________________________________________________
dense_73 (Dense)             (None, 512)               91648     
_________________________________________________________________
dense_74 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_75 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_76 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_77 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_78 (Dense)             (None, 32)              

### Prediction presentation

In [122]:
# np.array([X_test_seq[100]])
# prediction = model.predict(np.array([X_test_seq[100]]))

for numb in range(100, 140):
    print('Treść wypowiedzi:')
    print(X_test.iloc[numb])
    print('Klasyfikacja: ', y_test.iloc[numb])
    prediction = model.predict(np.array([X_test_seq_pad[numb]]))
    print('Predykcja: ', [round(prc, 2) for prc in prediction[0]])
    # print(X_test_seq_pad[numb])
    print('================================================')
    

Treść wypowiedzi:
['i', 'co', 'by', 'być', 'tutaj', 'nie', 'powiedzieć', 'złe', 'bo', 'zawsze', 'krytyk', 'się', 'znajda', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'wiele', 'to', 'Polska', 'w', 'moment', 'start', 'w', '89', 'rok', 'była', 'na', 'poziom', 'Białoruś', 'i', 'Ukraina', 'ukraina', 'czyli', 'przeciętny', 'Polak', 'wytwarzać', 'tyle', 'to', 'dochód', 'narodowy', 'ile', 'Białorusin', 'i', 'Ukrainiec', 'tam', 'minimalny', 'różnica', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'kilkadziesiąt', 'dolar'

# Evaluation